<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/S104SentenceAlignmentHunalignWSentenceTokeniser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiments with alignment tools

resources / corpora:
https://heibox.uni-heidelberg.de/d/d65daff8341e467c82b1/


Instructions are here:
https://github.com/danielvarga/hunalign


In [ ]:
!git clone https://github.com/danielvarga/hunalign.git

In [ ]:
%cd hunalign/src/hunalign
!pwd

/content/hunalign/src/hunalign
/content/hunalign/src/hunalign


In [ ]:
!make

In [ ]:
%cd /content/hunalign/
!pwd
!src/hunalign/hunalign data/hu-en.stem.dic examples/demo.hu.stem examples/demo.en.stem -hand=examples/demo.manual.ladder -text > /tmp/align.txt
# !less /tmp/align.txt

In [ ]:
!head --lines=20 /tmp/align.txt

In [ ]:
%cd /content/
!pwd

/content
/content


In [ ]:
# downloading files - en es
!wget https://heibox.uni-heidelberg.de/f/dc01904f8f9040a69610/?dl=1
!mv index.html?dl=1 solar-t01-en-RE01.txt
!wget https://heibox.uni-heidelberg.de/f/646957ccb3d24f359ed7/?dl=1
!mv index.html?dl=1 solar-t01-es-RE01.txt

In [ ]:
# downloading files - en de
!wget https://heibox.uni-heidelberg.de/f/dc489828a2d642bfba82/?dl=1
!mv index.html?dl=1 ab-t01-en.txt
# English file
!wget https://heibox.uni-heidelberg.de/f/e7e542d75d7644d1857d/?dl=1
!mv index.html?dl=1 ab-t01-de.txt
# Other language files

In [ ]:
!head --lines=20 ab-t01-en.txt

Review article
Blood Sugar Targets in Surgical Intensive Care
Management and Special Considerations in Patients With Diabetes
Dtsch Arztebl Int 2021; 118: 629-36. DOI: 10.3238/arztebl.m2021.0221
Roth, J; Sommerfeld, O; Birkenfeld, A L; Sponholz, C; Müller, U A; von Loeffelholz, C
ArticleAuthorsFigures & TablesReferencesMetrics
Background: 30–80% of patients being treated in intensive care units in the perioperative period develop hyperglycemia. This stress hyperglycemia is induced and maintained by inflammatory-endocrine and iatrogenic stimuli and generally requires treatment. There is uncertainty regarding the optimal blood glucose targets for patients with diabetes mellitus.

Methods: This review is based on pertinent publications retrieved by a selective search in PubMed and Google Scholar.

Results: Patients in intensive care with pre-existing diabetes do not benefit from blood sugar reduction to the same extent as metabolically healthy individuals, but they, too, are exposed to a 

In [ ]:
!head --lines=20 ab-t01-de.txt

In [ ]:
# detecting sentence boundaries
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

In [ ]:
# example for Armenian / Georgian sentence splitting if errors
# LSent = re.split('([:;]+)')
# .

In [ ]:
import re, os, sys
def sentTokenizer(FIn, FOut):
    for SPara in FIn:
        SPara = SPara.strip()
        if SPara == '': FOut.write('\n')
        all_sent = sent_tokenize(SPara)
        for SSent in all_sent:
            FOut.write(SSent + '\n')
    FOut.flush()

In [ ]:
FIn = open('ab-t01-en.txt', 'r')
FOut = open('ab-t01s-en.txt', 'w')

# FIn = open('ab-t01-de.txt', 'r')
# FOut = open('ab-t01s-de.txt', 'w')

In [ ]:
sentTokenizer(FIn, FOut)

In [ ]:
FIn = open('ab-t01-de.txt', 'r')
FOut = open('ab-t01s-de.txt', 'w')

In [ ]:
sentTokenizer(FIn, FOut)

In [ ]:
!./hunalign/src/hunalign/hunalign -text -realign -autodict=es2enAutodict.txt ./hunalign/data/null.dic solar-t01-en-RE01.txt solar-t01-es-RE01.txt > solar-t01-en2esAlignment.tsv

In [ ]:
# !./hunalign/src/hunalign/hunalign -text -realign -autodict=de2enAutodict.txt ./hunalign/data/null.dic ab-t01-en.txt ab-t01-de.txt > ab-t01-en2de.tsv
!./hunalign/src/hunalign/hunalign -text -realign -autodict=de2enAutodict.txt ./hunalign/data/null.dic ab-t01s-en.txt ab-t01s-de.txt > ab-t01s-en2de.tsv


In [ ]:
# copy dictionaries for editing
!cp es2enAutodict.txt es2enAutodict.dic
!cp de2enAutodict.txt de2enAutodict.dic

In [ ]:
# you can edit dictionaries and try realigning tests if results are not good, like in this example:
!./hunalign/src/hunalign/hunalign -text -realign -autodict=es2enAutodict2.txt es2enAutodict.txt solar-t01-en-RE01.txt solar-t01-es-RE01.txt > solar-t01-en2esAlignment2.txt

In [ ]:
%cd /content/

/content


# Converting tsv to tmx

In [ ]:
# change names of files accordingly (input and output)
import re, os, sys
# FIn = open('ab-t01-en2de.tsv', 'r')
# FOut = open('ab-t01-en2de.tmx', 'w')

FIn = open('ab-t01s-en2de.tsv', 'r')
FOut = open('ab-t01s-en2de.tmx', 'w')

SrcLang = 'en'
TgtLang = 'de'

SLineHead = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE tmx SYSTEM "tmx14.dtd">
<tmx version="1.4">
  <header creationtool="OmegaT" o-tmf="OmegaT TMX" adminlang="EN-US" datatype="plaintext" creationtoolversion="5.7.0_0_8ae1ecfb" segtype="sentence" srclang="en"/>
  <body>
'''
SLineEnd = '''
  </body>
</tmx>
'''

FOut.write(SLineHead)
for SLine in FIn:
    SLine = SLine.strip()
    LLine = re.split('\t', SLine)
    try:
        SSrcLang = LLine[0]
        STgtLang = LLine[1]

        STuvSLTag = f'      <tuv xml:lang="{SrcLang}">'
        STuvTLTag = f'      <tuv xml:lang="{TgtLang}">'

        STuvSL = STuvSLTag + '''\n        <seg>''' + SSrcLang + '''</seg>\n''' + '''      </tuv>\n'''
        STuvTL = STuvTLTag + '''\n        <seg>''' + STgtLang + '''</seg>\n''' + '''      </tuv>\n'''

        STuv = '''    <tu>\n''' + STuvSL + STuvTL + '''    </tu>\n'''
        FOut.write(STuv)
    except:
        continue
FOut.write(SLineEnd)
FOut.flush()